In [345]:
import numpy as np
import pandas as pd
import re
import datetime

# Code

In [346]:
data = 'https://www.ndbc.noaa.gov/data/realtime2/46029.spec'
article_read = pd.read_csv(data, sep='\s+', delimiter=' ', encoding="utf-8", skipinitialspace=True)

In [347]:
article_read.head()

,#YY,MM,DD,hh,mm,WVHT,SwH,SwP,WWH,WWP,SwD,WWD,STEEPNESS,APD,MWD
0,#yr,mo,dy,hr,mn,m,m,sec,m,sec,-,degT,-,sec,degT
1,2019,03,21,22,00,1.8,1.7,12.9,0.6,4.2,WSW,SSW,SWELL,7.2,243
2,2019,03,21,21,00,1.7,1.6,12.1,0.6,4.8,W,SSW,SWELL,7.2,280
3,2019,03,21,20,00,1.9,1.8,12.1,0.6,3.8,WSW,S,SWELL,7.4,245
4,2019,03,21,19,00,1.9,1.8,12.9,0.7,5.3,W,SW,SWELL,7.3,270


In [348]:
short_table = article_read.drop(article_read.iloc[:,3:6,], axis=1)

In [349]:
short_table = short_table.drop(['STEEPNESS', 'APD', 'MWD'], axis=1)

In [350]:
d = datetime.datetime.today().strftime('%Y-%m-%d')
d

'2019-03-21'

In [351]:
today_date = datetime.date.today()
today_date

datetime.date(2019, 3, 21)

In [352]:
short_table = short_table.rename(columns={'#YY': 'YY'})

In [353]:
date = (short_table.YY + '-' + short_table.MM + '-' + short_table.DD)
short_table['Date'] = date

In [354]:
data_table = short_table.drop(short_table.columns[[0,1,2]], axis=1)
data_table = data_table[data_table['Date']==d]

In [355]:
data_table['SwH'] = data_table.SwH.astype(float)
data_table['SwP'] = data_table.SwP.astype(float)
data_table['WWH'] = data_table.WWH.astype(float)
data_table['WWP'] = data_table.WWP.astype(float)

In [356]:
data_table.dtypes

SwH     float64
SwP     float64
WWH     float64
WWP     float64
SwD      object
WWD      object
Date     object
dtype: object

In [357]:
wave = data_table.SwH * data_table.SwP
data_table['Wave'] = wave

In [358]:
article_read.WWD.unique()

array(['degT', 'SSW', 'S', 'SW', 'ESE', 'SSE', 'SE', 'E', 'WSW', 'ENE',
       'N', 'W', 'NNE', 'WNW', 'NE', 'NW', 'NNW'], dtype=object)

In [359]:
wwd = pd.DataFrame({"WWD": [ 'SW', 'SSW', 'S', 'ESE', 'SSE', 'SE', 'E', 'WSW', 'ENE',
       'N', 'W', 'NNE', 'WNW', 'NE', 'NW', 'NNW' ], "scoreWWD": [1, 1, 3 ,1 ,1 ,4 ,5 ,1 ,1 ,1 ,1 ,1 , 1, 4,1,1]})
wwd.head()

,WWD,scoreWWD
0,SW,1
1,SSW,1
2,S,3
3,ESE,1
4,SSE,1


In [360]:
data_table =pd.merge(data_table, wwd)
data_table.head()

,SwH,SwP,WWH,WWP,SwD,WWD,Date,Wave,scoreWWD
0,1.7,12.9,0.6,4.2,WSW,SSW,2019-03-21,21.93,1
1,1.6,12.1,0.6,4.8,W,SSW,2019-03-21,19.36,1
2,1.8,10.8,0.6,3.7,W,SSW,2019-03-21,19.44,1
3,2.1,12.1,0.5,3.6,W,SSW,2019-03-21,25.41,1
4,2.1,12.1,0.5,5.0,W,SSW,2019-03-21,25.41,1


In [361]:
def scoreWave(column): 
    '''
    This is a test:
    >>> scoreWave(25)
    4
    '''
    
    if column > 30 :
        score = 5
    elif column >= 25 :
        score = 4
    elif column >= 20 :
        score = 3
    elif column >= 11 :
        score = 2
    else:
        score =1
    return score 

In [362]:
def scoreSwp(column):
    '''
    This is a test:
    >>> scoreSwp(12)
    4
    '''
        
    if column >= 16 :
        score = 5
    elif column >= 12 :
        score = 4
    elif column >= 10 :
        score = 4
    else:
        score =1
    return score 

In [363]:
data_table['scoreWave'] = data_table['Wave'].map(scoreWave)

In [364]:
data_table['scoreSwP'] = data_table['SwP'].map(scoreSwp)

In [365]:
data_table['Score'] = data_table.iloc[:, 8:11].sum(axis=1)

In [366]:
data_length = len(data_table)-1
data_length

22

In [367]:
data_table

,SwH,SwP,WWH,WWP,SwD,WWD,Date,Wave,scoreWWD,scoreWave,scoreSwP,Score
0,1.7,12.9,0.6,4.2,WSW,SSW,2019-03-21,21.93,1,3,4,8
1,1.6,12.1,0.6,4.8,W,SSW,2019-03-21,19.36,1,2,4,7
2,1.8,10.8,0.6,3.7,W,SSW,2019-03-21,19.44,1,2,4,7
3,2.1,12.1,0.5,3.6,W,SSW,2019-03-21,25.41,1,4,4,9
4,2.1,12.1,0.5,5.0,W,SSW,2019-03-21,25.41,1,4,4,9
5,2.0,13.8,0.5,4.5,W,SSW,2019-03-21,27.60,1,4,4,9
6,1.9,12.9,0.5,3.3,W,SSW,2019-03-21,24.51,1,3,4,8
7,1.9,12.9,0.8,3.3,W,SSW,2019-03-21,24.51,1,3,4,8
8,1.9,13.8,0.7,3.3,WNW,SSW,2019-03-21,26.22,1,4,4,9
9,2.1,13.8,0.3,4.5,W,SSW,2019-03-21,28.98,1,4,4,9


In [368]:
totalscore = data_table['Score'].sum()/(len(data_table)-1)
totalscore

9.318181818181818

In [369]:
day_score = round(totalscore)
day_score

9.0

# Test

The test is inside the function marked by ''' , this is space sensative so data must match the alighment.

In [382]:
data_length

22

In [383]:
data_table.apply(np.sum, axis=0)

SwH                                                       44.1
SwP                                                      291.4
WWH                                                       12.9
WWP                                                       98.5
SwD                    WSWWWWWWWWWNWWWSWWWSWWWSWWWWSWWNWWWWNWW
WWD          SSWSSWSSWSSWSSWSSWSSWSSWSSWSSWSSSSSSSWSWSWSWSW...
Date         2019-03-212019-03-212019-03-212019-03-212019-0...
Wave                                                    559.39
scoreWWD                                                    35
scoreWave                                                   78
scoreSwP                                                    92
Score                                                      205
dtype: object

In [384]:
def total(column): 
    '''
    This is a test:
    >>> total('Score') 
    205
    '''
    score = data_table[column].sum()
    return round(score)

In [385]:
def total_avg(column): 
    '''
    This is a test:
    >>> total_avg('Score') 
    9.0
    '''
    score = data_table[column].sum()/(data_length)
    return round(score)

In [386]:
total('Score')

205

In [387]:
total_avg('Score')

9.0

In [388]:
import doctest
doctest.testmod(verbose=True)

Trying:
    scoreSwp(12)
Expecting:
    4
ok
Trying:
    scoreWave(25)
Expecting:
    4
ok
Trying:
    total('Score') 
Expecting:
    205
ok
Trying:
    total_avg('Score') 
Expecting:
    9.0
ok
1 items had no tests:
    __main__
4 items passed all tests:
   1 tests in __main__.scoreSwp
   1 tests in __main__.scoreWave
   1 tests in __main__.total
   1 tests in __main__.total_avg
4 tests in 5 items.
4 passed and 0 failed.
Test passed.


TestResults(failed=0, attempted=4)

# UI